# Predict a biological response of molecules from their chemical properties
The objective of the competition is to help us build as good a model as possible so that we can, as optimally as this data allows, relate molecular information, to an actual biological response.

We have shared the data in the comma separated values (CSV) format. Each row in this data set represents a molecule. The first column contains experimental data describing an actual biological response; the molecule was seen to elicit this response (1), or not (0). The remaining columns represent molecular descriptors (d1 through d1776), these are calculated properties that can capture some of the characteristics of the molecule - for example size, shape, or elemental constitution. The descriptor matrix has been normalized.

The evaluation will be made by the use of the log loss function. Which will outpu y_i = 1 means that a molecule elicited a response, while y_i = 0 means it did not.

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
import scipy as sp
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from numpy import genfromtxt, savetxt

In [22]:
#log loss function
def llfun(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

In [65]:
# read data, parse into training and target sets
dataset = np.genfromtxt(open('Data\\train.csv','r'),delimiter=',',dtype='f8')[1:]
target = np.array([x[0] for x in dataset])
train = np.array([x[1:] for x in dataset])


## 1st classifier - Random forest

In [38]:
# Random forest classifier
cfr = RandomForestClassifier(n_estimators = 100, n_jobs=2)
# k-fold validation
cv = cross_validation.KFold(len(train), n_folds=5, shuffle = False)
#iterate through the training and test cross validation segments and
#run the classifier on each one, aggregating the results into a list
results = []
# for every value in rf.predict_proba(test) we will take the index and the x[1] values
predicted_probs = [[index + 1, x[1]] for index, x in enumerate(rf.predict_proba(test))]

## Results

In [49]:
print "Results: " + str(np.array([x[1] for x in predicted_probs]).mean())

Results: 0.551031587365


## 2nd classifier - MLP Neural network

In [62]:
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.tools.shortcuts import buildNetwork
from pybrain.structure.modules import *

In [71]:
# prepare a dataset
from pybrain.datasets import SupervisedDataSet
ds = SupervisedDataSet(len(train[0]),1)
i = 0
for itm in train:
    ds.addSample(itm,target[i])
    i += 1

In [78]:
# net = buildNetwork( input_size, hidden_size, target_size, bias = True )
net = buildNetwork(len(train[0]),3,1,bias=True,hiddenclass = TanhLayer)

In [82]:
trainer = BackpropTrainer(net,ds)
trainer.train()
trainer.trainUntilConvergence(verbose = True, validationProportion = 0.15, maxEpochs = 10, continueEpochs = 10)

train-errors: [  0.082934  0.083366  0.083043  0.083424  0.083160  0.082990  0.082701  0.082894  0.082653  0.082914  0.082185  0.082095]
valid-errors: [  0.077662  0.077534  0.076484  0.079085  0.077372  0.078357  0.080425  0.081373  0.077020  0.079138  0.078572  0.078785]


([0.082933589555442389,
  0.083365667199018473,
  0.083043232666128633,
  0.083424313337045572,
  0.083160359167587808,
  0.082990028099676261,
  0.082701383928152322,
  0.082893941153449446,
  0.08265290278323105,
  0.082914305639411282,
  0.082185469653545257,
  0.082095493173359277],
 [0.077661975646470788,
  0.077533613766263929,
  0.076484440866864359,
  0.079085075547347519,
  0.077371693729790164,
  0.078356811944768331,
  0.080424893658967447,
  0.081372628844643635,
  0.077019974284016163,
  0.079138209507172971,
  0.07857197111603452,
  0.078784961995478242])

In [103]:
out = net.activateOnDataset(ds)
print len(out)
res = []
i=1
for itm in out:
    res.append([i,itm[0]])
    i+=1
print res

3751
[[1, 0.79184373950226328], [2, 0.45491025585717265], [3, 0.10571044986357345], [4, 0.65048634852771414], [5, 0.099296159022876224], [6, 0.00728103656623863], [7, 0.81251075526674499], [8, 0.81253171395640233], [9, 0.81253331869400447], [10, 0.044047400809472337], [11, 0.78408060922586387], [12, 0.45413367203442612], [13, 0.50660412629409923], [14, 0.6977747630709128], [15, 0.60039380643582119], [16, 0.67852719206313528], [17, 0.81253149200096786], [18, 0.31301768556197196], [19, 0.56531541629277515], [20, 0.33108120089986448], [21, 0.81253407043941439], [22, 0.59870573399891658], [23, -0.01685988416967249], [24, 0.34075868945743859], [25, 0.21572305150811549], [26, 0.81253407042453141], [27, 0.81253406553509611], [28, 0.45491617179680266], [29, 0.81253406338901146], [30, 0.45491549790634978], [31, 0.34075659166538802], [32, 0.60731989401377962], [33, 0.099282963734553392], [34, 0.45491547396900345], [35, 0.81191493274780213], [36, 0.80839477799076098], [37, 0.34062626190951473], [

In [106]:
print "Results: " + str(np.array([x[1] for x in res]).mean())
print "Log loss: " + llfun(out,target)

Results: 0.515437874952


TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('S32') dtype('S32') dtype('S32')

## Generate Kaggle output

In [47]:
savetxt('Data/submission.csv', predicted_probs, delimiter=',', fmt='%d,%f', 
        header='MoleculeId,PredictedProbability', comments = '')

In [104]:
savetxt('Data/submission_NN.csv', res,delimiter=',',fmt='%d,%f',header="MoleculeId,PredictedProbability",comments = '')

# Other essays

In [111]:
from __future__ import division
import numpy as np
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

def read_data(file_name):
    """This function is taken from:
    https://github.com/benhamner/BioResponse/blob/master/Benchmarks/csv_io.py
    """
    f = open(file_name)
    #ignore header
    f.readline()
    samples = []
    target = []
    for line in f:
        line = line.strip().split(",")
        sample = [float(x) for x in line]
        samples.append(sample)
    return samples

def load():
    """Conveninence function to load all data as numpy arrays.
    """
    print "Loading data..."
    filename_train = 'data/train.csv'
    filename_test = 'data/test.csv'

    train = read_data("data/train.csv")
    y_train = np.array([x[0] for x in train])
    X_train = np.array([x[1:] for x in train])
    X_test = np.array(read_data("data/test.csv"))
    return X_train, y_train, X_test

def logloss(attempt, actual, epsilon=1.0e-15):
    """Logloss, i.e. the score of the bioresponse competition.
    """
    attempt = np.clip(attempt, epsilon, 1.0-epsilon)
    return - np.mean(actual * np.log(attempt) + (1.0 - actual) * np.log(1.0 - attempt))


if __name__ == '__main__':

    np.random.seed(0) # seed to shuffle the train set

    n_folds = 10
    verbose = True
    shuffle = False

    X, y, X_submission = load()

    if shuffle:
        idx = np.random.permutation(y.size)
        X = X[idx]
        y = y[idx]

    skf = list(StratifiedKFold(y, n_folds))

    clfs = [RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='gini'),
            RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='entropy'),
            ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='gini'),
            ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='entropy'),
            GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=50)]

    print "Creating train and test sets for blending."
    
    dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
    dataset_blend_test = np.zeros((X_submission.shape[0], len(clfs)))
    
    for j, clf in enumerate(clfs):
        print j, clf
        dataset_blend_test_j = np.zeros((X_submission.shape[0], len(skf)))
        for i, (train, test) in enumerate(skf):
            print "Fold", i
            X_train = X[train]
            y_train = y[train]
            X_test = X[test]
            y_test = y[test]
            clf.fit(X_train, y_train)
            y_submission = clf.predict_proba(X_test)[:,1]
            dataset_blend_train[test, j] = y_submission
            dataset_blend_test_j[:, i] = clf.predict_proba(X_submission)[:,1]
        dataset_blend_test[:,j] = dataset_blend_test_j.mean(1)

    print
    print "Blending."
    clf = LogisticRegression()
    clf.fit(dataset_blend_train, y)
    y_submission = clf.predict_proba(dataset_blend_test)[:,1]

    print "Linear stretch of predictions to [0,1]"
    y_submission = (y_submission - y_submission.min()) / (y_submission.max() - y_submission.min())

    print "Saving Results."
    np.savetxt(fname='test.csv', X=y_submission, fmt='%0.9f')

Loading data...
Creating train and test sets for blending.
0 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
1 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
2 ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',

In [113]:
res = []
i=1
for itm in y_submission:
    res.append([i,itm])
    i+=1
print res

[[1, 0.95009960021584183], [2, 0.96653980471624479], [3, 0.51125503296168318], [4, 0.99064969133153569], [5, 0.060057910731071126], [6, 0.62335064560432085], [7, 0.9766377116434749], [8, 0.77740148916927376], [9, 0.96564843149987623], [10, 0.44721316329354432], [11, 0.20295328555548239], [12, 0.80161550157046269], [13, 0.886464934801378], [14, 0.50885658489707331], [15, 0.22493018040789012], [16, 0.064456829155364284], [17, 0.099509746576914498], [18, 0.11570572016056248], [19, 0.35169687748990136], [20, 0.92740843288391117], [21, 0.063902840017690907], [22, 0.96414463953278817], [23, 0.93766140218410865], [24, 0.9828328872152301], [25, 0.95373408339459453], [26, 0.69922671653010526], [27, 0.053401296948953028], [28, 0.14972437144127967], [29, 0.074062648598233138], [30, 0.53711765299096359], [31, 0.19681896336604243], [32, 0.04716575868607277], [33, 0.85617608060516592], [34, 0.34237631065018942], [35, 0.31188664778371283], [36, 0.968823306654368], [37, 0.90645420505987706], [38, 0.83

In [114]:
savetxt('Data/submission_blend.csv', res,delimiter=',',fmt='%d,%f',header="MoleculeId,PredictedProbability",comments = '')